Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# import modules
import numpy as np
import tensorflow as tf
from six.moves import range
from __future__ import print_function
from six.moves import cPickle as pickle

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [ ]:
depth = 16
patch_size = 5
batch_size = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_test_dataset = tf.constant(test_dataset)
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    
    # Variables.
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  
    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    test_prediction = tf.nn.softmax(model(tf_test_dataset))
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))

In [ ]:
num_steps = 2501

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
        if (step % 500 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
  
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [5]:
depth1 = 8
depth2 = 12
patch_size = 5
batch_size = 32
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_test_dataset = tf.constant(test_dataset)
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    
    # Variables.
    layer1_biases = tf.Variable(tf.zeros([depth1]))
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth1], stddev=0.1))
    
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth2]))
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth1, depth2], stddev=0.1))
    
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer3_weights = tf.Variable(tf.truncated_normal([image_size//4*image_size//4*depth2, num_hidden], stddev=0.1))
  
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  
    # Model.
    ksize = [1,2,2,1]
    strides_conv_1 = [1,1,1,1]
    strides_conv_2 = [1,1,1,1]
    strides_pooling = [1,2,2,1]
    
    def model(data):
        # 3rd argument is the strides and we are setting stride to be 2
        conv = tf.nn.conv2d(data, layer1_weights, strides_conv_1, padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        pool = tf.nn.max_pool(hidden, ksize, strides_pooling, padding='SAME')
        conv = tf.nn.conv2d(pool, layer2_weights, strides_conv_2, padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        pool = tf.nn.max_pool(hidden, ksize, strides_pooling, padding='SAME')
        shape = pool.get_shape().as_list()
        reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    test_prediction = tf.nn.softmax(model(tf_test_dataset))
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))

In [7]:
num_steps = 5001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
        if (step % 500 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
  
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.823173
Minibatch accuracy: 6.2%
Validation accuracy: 10.6%
Minibatch loss at step 500: 1.000708
Minibatch accuracy: 75.0%
Validation accuracy: 82.2%
Minibatch loss at step 1000: 0.386703
Minibatch accuracy: 84.4%
Validation accuracy: 84.9%
Minibatch loss at step 1500: 0.298702
Minibatch accuracy: 93.8%
Validation accuracy: 86.2%
Minibatch loss at step 2000: 0.206223
Minibatch accuracy: 96.9%
Validation accuracy: 85.7%
Minibatch loss at step 2500: 0.392419
Minibatch accuracy: 87.5%
Validation accuracy: 87.4%
Minibatch loss at step 3000: 0.519320
Minibatch accuracy: 84.4%
Validation accuracy: 87.3%
Minibatch loss at step 3500: 0.324123
Minibatch accuracy: 84.4%
Validation accuracy: 87.8%
Minibatch loss at step 4000: 0.045295
Minibatch accuracy: 100.0%
Validation accuracy: 87.9%
Minibatch loss at step 4500: 0.547898
Minibatch accuracy: 81.2%
Validation accuracy: 87.9%
Minibatch loss at step 5000: 0.211477
Minibatch accuracy: 93.8%
Validation accurac

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [8]:
depth1 = 8
depth2 = 12
kprob = 0.6
patch_size = 5
batch_size = 32
num_hidden = 64

update_steps = 100
update_exponent = 0.96
initial_learning_rate = 6e-3

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_test_dataset = tf.constant(test_dataset)
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    
    # Variables.
    layer1_biases = tf.Variable(tf.zeros([depth1]))
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth1], stddev=0.1))
    
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth2]))
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth1, depth2], stddev=0.1))
    
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer3_weights = tf.Variable(tf.truncated_normal([image_size//4*image_size//4*depth2, num_hidden], stddev=0.1))
  
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  
    # Model.
    ksize = [1,2,2,1]
    strides_conv_1 = [1,1,1,1]
    strides_conv_2 = [1,1,1,1]
    strides_pooling = [1,2,2,1]
    keep_prob = tf.constant(kprob)
    
    def modelWithDrop(data):
        # 3rd argument is the strides and we are setting stride to be 2
        conv = tf.nn.conv2d(data, layer1_weights, strides_conv_1, padding='SAME')
        hidden = tf.nn.dropout(tf.nn.relu(conv + layer1_biases),keep_prob)
        pool = tf.nn.max_pool(hidden, ksize, strides_pooling, padding='SAME')
        conv = tf.nn.conv2d(pool, layer2_weights, strides_conv_2, padding='SAME')
        hidden = tf.nn.dropout(tf.nn.relu(conv + layer2_biases),keep_prob)
        pool = tf.nn.max_pool(hidden, ksize, strides_pooling, padding='SAME')
        shape = pool.get_shape().as_list()
        reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
    
    def modelWithoutDrop(data):
        # 3rd argument is the strides and we are setting stride to be 2
        conv = tf.nn.conv2d(data, layer1_weights, strides_conv_1, padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        pool = tf.nn.max_pool(hidden, ksize, strides_pooling, padding='SAME')
        conv = tf.nn.conv2d(pool, layer2_weights, strides_conv_2, padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        pool = tf.nn.max_pool(hidden, ksize, strides_pooling, padding='SAME')
        shape = pool.get_shape().as_list()
        reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
    # Training computation.
    logits = modelWithDrop(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(initial_learning_rate,global_step,update_steps,update_exponent)
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss,global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(modelWithoutDrop(tf_train_dataset))
    test_prediction = tf.nn.softmax(modelWithoutDrop(tf_test_dataset))
    valid_prediction = tf.nn.softmax(modelWithoutDrop(tf_valid_dataset))

In [9]:
num_steps = 5001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
        if (step % 500 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
  
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.197087
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 500: 0.951980
Minibatch accuracy: 75.0%
Validation accuracy: 83.3%
Minibatch loss at step 1000: 0.455644
Minibatch accuracy: 84.4%
Validation accuracy: 85.5%
Minibatch loss at step 1500: 0.291484
Minibatch accuracy: 93.8%
Validation accuracy: 86.4%
Minibatch loss at step 2000: 0.293169
Minibatch accuracy: 90.6%
Validation accuracy: 86.6%
Minibatch loss at step 2500: 0.499476
Minibatch accuracy: 90.6%
Validation accuracy: 87.3%
Minibatch loss at step 3000: 0.539029
Minibatch accuracy: 81.2%
Validation accuracy: 87.6%
Minibatch loss at step 3500: 0.259914
Minibatch accuracy: 96.9%
Validation accuracy: 88.3%
Minibatch loss at step 4000: 0.101242
Minibatch accuracy: 100.0%
Validation accuracy: 88.3%
Minibatch loss at step 4500: 0.748923
Minibatch accuracy: 78.1%
Validation accuracy: 88.4%
Minibatch loss at step 5000: 0.199577
Minibatch accuracy: 96.9%
Validation accura